# 問題文
高橋君は食べ放題のお店に来ました。

$N$ 種類の料理があり、$i$ 番目の料理は、食べるために $A_i$ 分必要で、美味しさは $B_i$ です。
<br>
<br>
この店のルールは以下の通りです。

$1$ 度に $1$ つの料理のみを注文することができます。注文した料理は即座に提供され、食べ始めることができます。

同じ種類の料理を $2$ 度以上注文することはできません。

提供済みの料理を食べ終わるまで次の料理を注文することはできません。

最初の注文から $T - 0.5$ 分後以降に注文することはできませんが、提供済みの料理を食べることはできます。

高橋君の満足度を、この来店で高橋君が食べる料理の美味しさの合計とします。

高橋君が適切に行動したとき、満足度は最大でいくらになるでしょうか。

# 制約

- $2 \leq N \leq 3000$
- $1 \leq T \leq 3000$
- $1 \leq A_i \leq 3000$
- $1 \leq B_i \leq 3000$

入力中のすべての値は整数である。

### 文章読解
Let $I = \{k_1, k_2, \cdots, k_n\}$ and $I^\prime = I  \backslash \{k_n\}$.<br>
Then, we can write the optimization problem as follow;

$$
    \begin{align}
        maximize \sum_{i \in I} B_i \\
        subject\ to\ \sum_{i \in I^\prime} A_i < T - 0.5
    \end{align}
$$

## ナイーブに実装してみる

In [77]:
N = 10 
T = 100
array = [
    [15, 23],
    [20, 18],
    [13, 17],
    [24, 12],
    [18, 29],
    [19, 27],
    [23, 21],
    [18, 20],
    [27, 15],
    [22, 25]
]

In [80]:
import itertools
import numpy as np


N, T = map(int, input().split())
array = []
for _ in range(N):
    array.append([int(i) for i in input().split()])


# bの昇順でソート 
a = np.array(array)
a = a[np.argsort(a[:, 1])]

# 制限時間内に注文できる全ての組を列挙し、残りの中から最も大きい効用が得られるものを注文する
candidate = []
_range = [i for i in range(N)]
for i in range(1, N+1):
    for j in itertools.combinations(_range, i):
        j = np.array(j)
        if a[j].sum(0)[0] < T:
            sum_A = a[j].sum(0)[1]
            for k in reversed(range(N)):
                if k not in j:
                    break
                else:
                    continue
            ret = a[j].sum(0)[1] + a[k, 1]
            candidate.append(ret)
print(max(candidate))

KeyboardInterrupt: 

### ビビるほどTLEした 枝刈り必須
- コスト順に並べてn個選んだコストの最小値を評価することでコンビネーションの回数が減らせる

In [89]:
import itertools
import numpy as np


N, T = map(int, input().split())
a = []
b = []
for _ in range(N):
    _a, _b = map(int, input().split())
    a.append(_a)
    b.append(_b)
a = np.array(a)
b = np.array(b)
    
costs = a[np.argsort(-a)]
cost_cumsum = np.cumsum(costs)
costs_value_sort = a[np.argsort(-b)]

values = b[np.argsort(-b)]
values_cost_sort = b[np.argsort(-a)]

# 下限の計算
index = np.sum(np.cumsum(costs) < T) - 1
if len(costs) == index + 1:
    inf = np.cumsum(values)
else:
    inf = np.cumsum(values_cost_sort)[index+1]
    
candidate = []
_range = [i for i in range(N)]

for i in range(0, N):
    if cost_cumsum[i] > T - 1:
        continue
    else:
        break

if N - i == 1:
    print(values.sum())
    
else:
    for j in range(1, N - i):
        for k in itertools.combinations(_range, j):
            k = np.array(k)
            current_cost = costs_value_sort[k].sum()
            if current_cost < T:
                for l in range(N):
                    if l not in k:
                        break
                    else:
                        continue
                ret = values[k].sum() + values[l]
            elif current_cost == T:
                ret = values[k].sum()
            else:
                continue
                
            if inf < ret:
                inf = ret
    print(inf)

KeyboardInterrupt: 

In [87]:
# リストを受け取って次に足しこむ
def cumsum(list_object):
    N = len(list_object)
    ret = [0] * N
    for i in range(N):
        ret[i] = ret[i-1] + list_object[i]
    return ret

In [92]:
np.arange(10)[9:]

array([9])

## 思いついたこと

### 枝刈り
- 満足度の下限を使って枝刈りをする
    - $max(B_i)$ が満足度の下限になる
    - $A_{argmax{B_i}} \leq T - 1$ なら満足度2番目を満足度1番目に足しこんだものが満足度の下限になる。以下同様。
- 残り時間が $t$ の時、$min(A_i) > t$ ならば、その時点で残っている満足度最大のものを注文するのがoptimal

In [1]:
import numpy as np


N, T = map(int, input().split())
array = []
for _ in range(N):
    array.append([int(i) for i in input().split()])

3 60
10 10
10 20
10 30


In [14]:
N = 10 
T = 100
array = [
    [15, 23],
    [20, 18],
    [13, 17],
    [24, 12],
    [18, 29],
    [19, 27],
    [23, 21],
    [18, 20],
    [27, 15],
    [22, 25]
]

In [15]:
a = np.array(array)
a

array([[15, 23],
       [20, 18],
       [13, 17],
       [24, 12],
       [18, 29],
       [19, 27],
       [23, 21],
       [18, 20],
       [27, 15],
       [22, 25]])

In [16]:
index = np.argsort(a[:, 0])
index

array([2, 0, 4, 7, 5, 1, 9, 6, 3, 8])

In [24]:
count = 0
sum_A = 0
while T - 1 >= sum_A:
    sum_A += a[index[count], 1]
    print(a[index[count], 1])
    count += 1
res_max = np.max(a[index[count:], 1])
ret = sum_A + res_max

17
23
29
20
27


In [18]:
sum_A

116

In [25]:
ret

141

In [26]:
count

5

In [23]:
index[:count]

array([2, 0, 4, 7, 5])

In [27]:
index[count:]

array([1, 9, 6, 3, 8])

### 比較演算子
- 第一引数が大きければ True を返す
- それ以外の場合は第一引数が小さいあるいは判定不能で何も返さない

In [32]:
def compare(a, b):
    # a.shape = b.shape = (2,)
    if a[0] <= b[0] and a[1] >= b[1]:
        return True

In [ ]:
import itertools

itertools.combinations(index)

In [31]:
a[0][0]

15

In [73]:
candidate

[41,
 44,
 46,
 47,
 49,
 50,
 52,
 54,
 56,
 56,
 56,
 58,
 59,
 61,
 62,
 64,
 66,
 68,
 68,
 61,
 62,
 64,
 65,
 67,
 69,
 71,
 71,
 64,
 66,
 67,
 69,
 71,
 73,
 73,
 67,
 68,
 70,
 72,
 74,
 74,
 70,
 72,
 74,
 76,
 76,
 73,
 75,
 77,
 77,
 77,
 79,
 79,
 81,
 81,
 81,
 73,
 74,
 76,
 77,
 79,
 81,
 83,
 83,
 76,
 78,
 79,
 81,
 83,
 85,
 85,
 79,
 80,
 82,
 84,
 86,
 86,
 82,
 84,
 86,
 88,
 88,
 85,
 87,
 89,
 89,
 89,
 91,
 91,
 93,
 93,
 93,
 79,
 81,
 82,
 84,
 86,
 88,
 88,
 82,
 83,
 85,
 87,
 89,
 89,
 85,
 87,
 89,
 91,
 91,
 88,
 90,
 92,
 92,
 92,
 94,
 94,
 96,
 96,
 96,
 84,
 85,
 87,
 89,
 91,
 91,
 87,
 89,
 91,
 93,
 93,
 90,
 92,
 94,
 94,
 94,
 96,
 96,
 98,
 98,
 98,
 88,
 90,
 92,
 94,
 94,
 91,
 93,
 95,
 95,
 95,
 97,
 97,
 99,
 99,
 99,
 93,
 95,
 97,
 97,
 97,
 99,
 99,
 101,
 101,
 101,
 98,
 100,
 100,
 102,
 102,
 102,
 104,
 104,
 104,
 104,
 91,
 93,
 94,
 96,
 98,
 100,
 100,
 94,
 95,
 97,
 99,
 101,
 101,
 97,
 99,
 101,
 103,
 103,
 100,
 102,
 104

In [55]:
np.array(j)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [59]:
_j = (0,)
a[np.array(_j)]

array([[15, 23]])

In [60]:
a[np.array(_j)].sum(0)

array([15, 23])

In [61]:
a[np.array(_j)].sum(0)[1]

15

In [67]:
a = np.array(array)
a = a[np.argsort(a[:, 1])]
a

array([[24, 12],
       [27, 15],
       [13, 17],
       [20, 18],
       [18, 20],
       [23, 21],
       [15, 23],
       [22, 25],
       [19, 27],
       [18, 29]])

In [66]:
np.array(array)

array([[15, 23],
       [20, 18],
       [13, 17],
       [24, 12],
       [18, 29],
       [19, 27],
       [23, 21],
       [18, 20],
       [27, 15],
       [22, 25]])

In [71]:
1 not in np.array([1, 2])

False